In [31]:
# Imports

# File handling
import os
import pathlib
from zipfile import ZipFile
#import splitfolders

# Visualization
import  matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

# Data and maths
import numpy as np
import pandas as pd
#import random

# ML
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.utils import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
#from sklearn.model_selection import train_test_split
#from keras.callbacks import CSVLogger # if problems with SAVING model

# Metrics
#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [19]:
# image data
splitfolders.ratio("data/images_unf/", output="images", seed=586, ratio=(.8, .1, .1), group_prefix=None, move=True)

Copying files: 21165 files [01:16, 277.95 files/s]
Copying files: 0 files [42:48, ? files/s]

model 1: loss: 0.6931 - accuracy: 0.4816 - val_loss: 0.6931 - val_accuracy: 0.4818  

model 2: loss: 8.3040 - accuracy: 0.4834 (interrupted)  

model 3: loss: 7.7412 - accuracy: 0.5178 - val_loss: 7.7656 - val_accuracy: 0.5182 (interrupted)  

model 4: loss: 0.7825 - accuracy: 0.4816 - val_loss: 0.6931 - val_accuracy: 0.4818  

model 5: loss: 0.4360 - accuracy: 0.8014 - val_loss: 0.4345 - val_accuracy: 0.8052  

model 6: loss: 0.4879 - accuracy: 0.7790 - val_loss: 0.4714 - val_accuracy: 0.7721  

model 7: loss: 0.3973 - accuracy: 0.8236 - val_loss: 0.4227 - val_accuracy: 0.8132  

model 8: loss: 0.0499 - accuracy: 0.9820 - val_loss: 0.6409 - val_accuracy: 0.8771  

model 14: loss: 0.7177 - accuracy: 0.5157 - val_loss: 0.7177 - val_accuracy: 0.5182  

model15 : loss: 0.7157 - accuracy: 0.4834 (interrupted)  

model 16: loss: 0.4366 - accuracy: 0.8351 - val_loss: 0.4091 - val_accuracy: 0.8459 (50 epochs)  

model 17: loss: 0.5579 - accuracy: 0.7285 - val_loss: 0.5146 - val_accuracy: 0.7688 (2 epochs!!)  

model 18: loss: 0.4710 - binary_accuracy: 0.7944 - val_loss: 0.4305 - val_binary_accuracy: 0.8170 (2 epochs!!!)  # increased learning rate, data augumentation, resizing, l2 regulariztion and binary classification

model 19: loss: 0.5588 - binary_accuracy: 0.7324 - val_loss: 0.4958 - val_binary_accuracy: 0.7759 (2 epochs!)  


In [2]:
# preprocess train, test and valid sets
train_path = 'images/train/'
test_path = 'images/test/'
valid_path = 'images/val/'
size = (256, 256)

train = image_dataset_from_directory(directory=train_path,
                         labels='inferred',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=True,
                         seed=77)

test = image_dataset_from_directory(directory=test_path,
                         labels='inferred',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=False,
                         seed=77)

valid = image_dataset_from_directory(directory=valid_path,
                         labels='inferred',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=False,
                         seed=77)

Found 16930 files belonging to 2 classes.
Found 2120 files belonging to 2 classes.
Found 2115 files belonging to 2 classes.


In [3]:
def evaluate_model(model, val=valid, test=test):

    print('Learning curve plot')
    plt.subplots(1, figsize=(15,5))
    plt.plot(model.history.history['accuracy'], color="#111111",  label="Training score")

    plt.title("Learning Curve")
    plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
    plt.tight_layout()
    #plt.show()

    print()
    print()

    print('Accuracy over validation set')
    plt.plot(model.history.history['accuracy'], color="#111111",  label="Training score")
    plt.plot(model.history.history['val_accuracy'])
    plt.title('Model Accuracy over Validation Set')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'])
    #plt.show()

    print()
    print()

    print('Test set score')
    model.evaluate(test)

    print()
    print()
    
    print('Validation set score')
    model.evaluate(val)

    print()
    print()

    print('Confusion matrix')
    classes = ['healthy', 'sick']
    y_actual = test.labels
    pred = model.predict(test)
    y_pred = np.argmax(pred,axis=1)
    sns.heatmap(confusion_matrix(y_actual, y_pred), yticklabels=classes, xticklabels=classes)

    print()
    print()

    print('Classification report')
    print(classification_report(y_actual,y_pred))

In [3]:
input_shape = (256, 256, 1)
num_classes=2

model1 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=input_shape),
                         
        layers.Conv2D(16, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(pool_size=(2, 2)),
                         
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Flatten(), # Dimensionality reduction (feature map)
        layers.Dropout(0.5), # Output layer
        layers.Dense(num_classes) # fully connected layer
    ])

model1.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        1

In [28]:
input_shape = (256, 256, 1)
num_classes=2

model2 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=input_shape),
                         
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Flatten(), # Dimensionality reduction (feature map)
        layers.Dropout(0.5), # Output layer
        layers.Dense(num_classes) # fully connected layer
    ])

model2.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_10 (Rescaling)    (None, 256, 256, 1)       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 246016)           

In [39]:
input_shape = (256, 256, 1)
num_classes=2

model3 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=input_shape),
                         
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Flatten(), # Dimensionality reduction (feature map)
        layers.Dropout(0.5), # Output layer
        layers.Dense(num_classes) # fully connected layer
    ])

model3.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_14 (Rescaling)    (None, 256, 256, 1)       0         
                                                                 
 conv2d_28 (Conv2D)          (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 246016)          

In [43]:
input_shape = (256, 256, 1)
num_classes=2

model4 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=input_shape),
                         
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Conv2D(128, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Flatten(), # Dimensionality reduction (feature map)
        layers.Dropout(0.5), # Output layer
        layers.Dense(num_classes) # fully connected layer
    ])

model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model4.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_16 (Rescaling)    (None, 256, 256, 1)       0         
                                                                 
 conv2d_32 (Conv2D)          (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 60, 60, 128)     

In [56]:
shape = (256, 256, 3)
num_classes=2

model5 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.Flatten(),
        
        layers.Dense(300, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(100, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="sigmoid") # output
    ])

model5.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model5.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_13 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 flatten_13 (Flatten)        (None, 196608)            0         
                                                                 
 dense_45 (Dense)            (None, 300)               58982700  
                                                                 
 flatten_14 (Flatten)        (None, 300)               0         
                                                                 
 dense_46 (Dense)            (None, 100)               30100     
                                                                 
 flatten_15 (Flatten)        (None, 100)               0         
                                                                 
 dense_47 (Dense)            (None, 2)               

In [60]:
shape = (256, 256, 3)
num_classes=2

model6 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.Flatten(),
        
        layers.Dense(300, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(100, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="softmax") # output
    ])

model6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model6.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_14 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 flatten_16 (Flatten)        (None, 196608)            0         
                                                                 
 dense_48 (Dense)            (None, 300)               58982700  
                                                                 
 flatten_17 (Flatten)        (None, 300)               0         
                                                                 
 dense_49 (Dense)            (None, 100)               30100     
                                                                 
 flatten_18 (Flatten)        (None, 100)               0         
                                                                 
 dense_50 (Dense)            (None, 2)               

In [ ]:
shape = (256, 256, 3)
num_classes=2

model7 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.Flatten(),
        
        layers.Dense(200, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(100, activation="relu"), # layer
        layers.Flatten(),

        layers.Dense(100, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="softmax") # output
    ])

model7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
shape = (256, 256, 3)
num_classes=2

model8 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.MaxPooling2D(2),
     
        layers.Conv2D(200, kernel_size=(3, 3), activation="relu"), # layer
        layers.MaxPooling2D(2),
        
        layers.Conv2D(150, kernel_size=(3, 3), activation="relu"), # layer
        layers.MaxPooling2D(2),

        layers.Conv2D(50, kernel_size=(3, 3), activation="relu"), # layer
        layers.MaxPooling2D(2),
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="softmax") # output
    ])

model8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
shape = (256, 256, 3)
drop = 0.4
kernel = (2, 2)
pool = (2, 2)
num_classes=2
regularizer=keras.regularizers.l1(0.01)


model10 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.MaxPooling2D(pool_size=pool),
     
        layers.Conv2D(200, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
        
        layers.Conv2D(50, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),

     
        layers.Flatten(),
        layers.Dropout(drop),     
        layers.Dense(num_classes) # output
    ])

model10.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model10.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 128, 128, 3)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 127, 127, 200)     2600      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 63, 200)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 62, 62, 50)        40050     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 31, 31, 50)      

In [7]:
model10.fit(train, validation_data=valid, verbose=1, epochs=20)

Epoch 1/20
116/530 [=====>........................] - ETA: 4:33 - loss: 5.6711 - accuracy: 0.5259

KeyboardInterrupt: 

In [7]:
shape = (256, 256, 3)
drop = 0.5
kernel = (3, 3)
pool = (2, 2)
num_classes=2
regularizer=keras.regularizers.l1(0.01)


model14 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.MaxPooling2D(pool_size=pool),
     
        layers.Conv2D(64, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
        
        layers.Conv2D(32, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),

     
        layers.Flatten(),
        layers.Dropout(drop),     
        layers.Dense(num_classes, actication="softmax") # output
    ])

model14.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model14.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 128, 128, 3)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 63, 63, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 32)        18464     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 30, 30, 32)      

In [8]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model14.fit(train, validation_data=valid, verbose=1, epochs=50, shuffle=True, callbacks=[early_stopping_cb])

Epoch 1/50
530/530 [==============================] - 149s 280ms/step - loss: 1.5506 - accuracy: 0.5141 - val_loss: 0.7199 - val_accuracy: 0.5182
Epoch 2/50
530/530 [==============================] - 149s 281ms/step - loss: 0.7170 - accuracy: 0.5172 - val_loss: 0.7167 - val_accuracy: 0.5182
Epoch 3/50
530/530 [==============================] - 147s 277ms/step - loss: 0.7170 - accuracy: 0.5182 - val_loss: 0.7174 - val_accuracy: 0.5182
Epoch 4/50
530/530 [==============================] - 146s 275ms/step - loss: 0.7173 - accuracy: 0.5178 - val_loss: 0.7172 - val_accuracy: 0.5182
Epoch 5/50
530/530 [==============================] - 146s 276ms/step - loss: 0.7175 - accuracy: 0.5165 - val_loss: 0.7174 - val_accuracy: 0.5182
Epoch 6/50
530/530 [==============================] - 147s 278ms/step - loss: 0.7176 - accuracy: 0.5183 - val_loss: 0.7177 - val_accuracy: 0.5182
Epoch 7/50
530/530 [==============================] - 150s 282ms/step - loss: 0.7177 - accuracy: 0.5157 - val_loss: 0.7177 -

In [26]:
shape = (256, 256, 3)
drop = 0.5
kernel = (3, 3)
pool = (2, 2)
num_classes=2
regularizer=keras.regularizers.l1(0.01)


model15 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.MaxPooling2D(pool_size=pool),
        
        layers.Conv2D(32, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
        layers.Dropout(drop),
        
        layers.Conv2D(32, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
        layers.Dropout(drop),
        
        layers.Conv2D(32, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
     
        layers.Flatten(),
        layers.Dropout(drop),     
        layers.Dense(num_classes) # output
    ])
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001, decay=1e-4)
model15.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model15.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_15 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 max_pooling2d_51 (MaxPoolin  (None, 128, 128, 3)      0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_52 (MaxPoolin  (None, 63, 63, 32)       0         
 g2D)                                                            
                                                                 
 dropout_18 (Dropout)        (None, 63, 63, 32)        0         
                                                                 
 conv2d_37 (Conv2D)          (None, 61, 61, 32)      

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model15.fit(train, validation_data=valid, verbose=1, epochs=50, shuffle=True, callbacks=[early_stopping_cb])

In [29]:
shape = (256, 256, 3)
drop = 0.5
kernel = (3, 3)
pool = (2, 2)
num_classes=2
regularizer=keras.regularizers.l1(0.01)


model16 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.MaxPooling2D(pool_size=pool),
     
        layers.Conv2D(64, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
        
        layers.Conv2D(32, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),

     
        layers.Flatten(),
        layers.Dropout(drop),     
        layers.Dense(num_classes, activation="softmax") # output
    ])

model16.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model16.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_17 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 max_pooling2d_58 (MaxPoolin  (None, 128, 128, 3)      0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d_59 (MaxPoolin  (None, 63, 63, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 61, 61, 32)        18464     
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 30, 30, 32)     

In [30]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model16.fit(train, validation_data=valid, verbose=1, epochs=50, shuffle=True, callbacks=[early_stopping_cb])

Epoch 1/50
530/530 [==============================] - 151s 284ms/step - loss: 1.1508 - accuracy: 0.7456 - val_loss: 0.5558 - val_accuracy: 0.7400
Epoch 2/50
530/530 [==============================] - 148s 279ms/step - loss: 0.5366 - accuracy: 0.7741 - val_loss: 0.4956 - val_accuracy: 0.8005
Epoch 3/50
530/530 [==============================] - 147s 277ms/step - loss: 0.5193 - accuracy: 0.7853 - val_loss: 0.4773 - val_accuracy: 0.8123
Epoch 4/50
530/530 [==============================] - 146s 274ms/step - loss: 0.5089 - accuracy: 0.7901 - val_loss: 0.4731 - val_accuracy: 0.8213
Epoch 5/50
530/530 [==============================] - 146s 276ms/step - loss: 0.5001 - accuracy: 0.7980 - val_loss: 0.4634 - val_accuracy: 0.8222
Epoch 6/50
530/530 [==============================] - 146s 275ms/step - loss: 0.4955 - accuracy: 0.7996 - val_loss: 0.4622 - val_accuracy: 0.8165
Epoch 7/50
530/530 [==============================] - 146s 275ms/step - loss: 0.4925 - accuracy: 0.8012 - val_loss: 0.4566 -

In [32]:
model16.save("model16.h5")

In [35]:
shape = (256, 256, 3)
drop = 0.6
kernel = (3, 3)
pool = (2, 2)
num_classes=2
regularizer=keras.regularizers.l2(0.01)


model17 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape),
        layers.Resizing(128, 128), # input
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.MaxPooling2D(pool_size=pool),
     
        layers.Conv2D(64, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
        
        layers.Conv2D(32, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),

     
        layers.Flatten(),
        layers.Dropout(drop),     
        layers.Dense(num_classes, activation="softmax") # output
    ])

model17.compile(optimizer=Adam(learning_rate=0.00164),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model17.summary()

early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model17.fit(train, validation_data=valid, verbose=1, epochs=2, shuffle=True, callbacks=[early_stopping_cb])

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_19 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 resizing_1 (Resizing)       (None, 128, 128, 3)       0         
                                                                 
 random_flip_1 (RandomFlip)  (None, 128, 128, 3)       0         
                                                                 
 random_rotation_1 (RandomRo  (None, 128, 128, 3)      0         
 tation)                                                         
                                                                 
 max_pooling2d_64 (MaxPoolin  (None, 64, 64, 3)        0         
 g2D)                                                            
                                                                 
 conv2d_45 (Conv2D)          (None, 62, 62, 64)      

In [43]:
# preprocess train, test and valid sets
train_path = 'images/train/'
test_path = 'images/test/'
valid_path = 'images/val/'
size = (256, 256)

train = image_dataset_from_directory(directory=train_path,
                         labels='inferred',
                         label_mode='binary',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=True,
                         seed=77)

test = image_dataset_from_directory(directory=test_path,
                         labels='inferred',
                         label_mode='binary',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=False,
                         seed=77)

valid = image_dataset_from_directory(directory=valid_path,
                         labels='inferred',
                         label_mode='binary',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=False,
                         seed=77)

shape = (256, 256, 3)
drop = 0.6
kernel = (3, 3)
pool = (2, 2)
num_classes=2
regularizer=keras.regularizers.l2(0.01)


model18 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.MaxPooling2D(pool_size=pool),
     
        layers.Conv2D(64, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
        
        layers.Conv2D(32, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),

     
        layers.Flatten(),
        layers.Dropout(drop),     
        layers.Dense(1, activation="sigmoid") # output
    ])

model18.compile(optimizer=Adam(learning_rate=0.0017),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

model18.summary()
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model18.fit(train, validation_data=valid, verbose=1, epochs=2, shuffle=True, callbacks=[early_stopping_cb])

Found 16930 files belonging to 2 classes.
Found 2120 files belonging to 2 classes.
Found 2115 files belonging to 2 classes.
Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_23 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 max_pooling2d_76 (MaxPoolin  (None, 128, 128, 3)      0         
 g2D)                                                            
                                                                 
 conv2d_53 (Conv2D)          (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d_77 (MaxPoolin  (None, 63, 63, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 61, 61, 32)        18464 

In [41]:
# preprocess train, test and valid sets
train_path = 'images/train/'
test_path = 'images/test/'
valid_path = 'images/val/'
size = (256, 256)

train = image_dataset_from_directory(directory=train_path,
                         labels='inferred',
                         label_mode='binary',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=True,
                         seed=77)

test = image_dataset_from_directory(directory=test_path,
                         labels='inferred',
                         label_mode='binary',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=False,
                         seed=77)

valid = image_dataset_from_directory(directory=valid_path,
                         labels='inferred',
                         label_mode='binary',
                         color_mode="rgb",
                         image_size=size,
                         shuffle=False,
                         seed=77)

shape = (256, 256, 3)
drop = 0.6
kernel = (3, 3)
pool = (2, 2)
num_classes=2
regularizer=keras.regularizers.l2(0.01)


model19 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.Resizing(128, 128),
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.MaxPooling2D(pool_size=pool),
     
        layers.Conv2D(64, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),
        
        layers.Conv2D(32, kernel_size=kernel, kernel_regularizer=regularizer, activation="relu"), # layer
        layers.MaxPooling2D(pool_size=pool),

     
        layers.Flatten(),
        layers.Dropout(drop),     
        layers.Dense(1, activation="sigmoid") # output
    ])

model19.compile(optimizer=Adam(learning_rate=0.00164),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

Found 16930 files belonging to 2 classes.
Found 2120 files belonging to 2 classes.
Found 2115 files belonging to 2 classes.


In [42]:
model19.summary()
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model19.fit(train, validation_data=valid, verbose=1, epochs=2, shuffle=True, callbacks=[early_stopping_cb])

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_22 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 resizing_3 (Resizing)       (None, 128, 128, 3)       0         
                                                                 
 random_flip_3 (RandomFlip)  (None, 128, 128, 3)       0         
                                                                 
 random_rotation_3 (RandomRo  (None, 128, 128, 3)      0         
 tation)                                                         
                                                                 
 max_pooling2d_73 (MaxPoolin  (None, 64, 64, 3)        0         
 g2D)                                                            
                                                                 
 conv2d_51 (Conv2D)          (None, 62, 62, 64)      